In [1]:
import spacy

In [2]:
def relative_clause(sentence):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sentence)
    rel = False
    rec=False
    list_rel = ['relcl', 'advcl']
    sent = [token for token in doc]
    r_clause = []
    place = 0

    for token in doc:
        #print(token.i,token.text,token.pos_, token.tag_,token.head.i,token.head,token.dep_)
        if not rel:
            if token.tag_ == 'WDT' or token.tag_ == 'WP':
                print('WDT')
                if doc[token.i - 1].pos_ == 'PUNCT':
                    sent.remove(doc[token.i - 1])
                h = token.head
                sub = [elt for elt in h.subtree]
                if ((len(doc) - 1) > sub[len(sub) - 1].i and sub[0].i != 0)and(doc[sub[0].i - 1].text and doc[sub[len(sub) - 1].i + 1].text) == ',':
                    print(1)
                    sent = [elt for elt in sent if (elt not in sub) and (elt != doc[sub[len(sub) - 1].i + 1])]
                    rel = True
                elif h.tag_ == 'VBZ' and h.head.tag_ == 'VBZ':
                    print(2)
                    place = h.head.i
                    idx = sent.index(h.head)
                    for elt in h.subtree:
                        sent.remove(elt)
                        if elt.i < h.i:
                            if elt != token and elt.tag_ != 'PRP':
                                r_clause.append(elt)
                        else:
                            r_clause.append(elt)
                    r_clause.append(nlp('to')[0])
                    sent[idx] = nlp(h.head.lemma_)[0]
                    rel=True


                elif h.lemma_ == 'be' and h.head.tag_ in ('NN', 'NNS') :
                    print(3)
                    cat=[elt for elt in h.children if elt.i> h.i]
                    if cat[0].dep_=='advmod':
                        text = doc.text
                        return (r_clause, sent, place, text)
                    if doc[h.head.i - 1].tag_ == 'DT':
                        place = h.head.i
                    elif doc[h.head.i - 1].tag_ == 'JJ':
                        place = h.head.i - 1
                    for elt in h.subtree:
                        sent.remove(elt)
                        if elt != token and elt != h:
                            r_clause.append(elt)
                    rel=True

                elif h.pos_ == 'VERB' and h.head.tag_ in ('NN', 'NNS'):
                    if len(doc)-1>h.i and doc[h.i+1].lemma_=='be':
                        print(4)
                        r_clause=[elt for i,elt in enumerate(h.subtree) if i!=0]
                        t=""
                        for elt in doc[h.i+1].subtree:
                            if elt not in r_clause:
                                t+=elt.text+' '
                        a,sent,b,t= relative_clause(t)

                    elif (len(doc)-1==sub[-1].i or (doc[len(doc)-1].pos_=='punct'and doc[len(doc)-2]==sub[-1]) ) and sub[-1].pos_=="VERB" and token.tag_=='WDT':
                        print(5)
                        text =doc.text
                        return (r_clause, sent, place, text)

                    else:
                        print(6)
                        subj = [elt for elt in h.head.subtree if elt not in sub]
                        subj[0] = nlp(subj[0].text.capitalize())[0]
                        for i, elt in enumerate(sub):
                            sent.remove(elt)
                            if i != 0:
                                r_clause.append(elt)
                        r_clause = subj + r_clause
                        r_clause.append(nlp('.')[0])
                        if sent[len(sent) - 1].pos_ != 'PUNCT':
                            sent.append(nlp('.')[0])

                        place = len(sent)
                    rel = True
                else:
                    print(7)
                    if sub[-1].i==doc[-1].i or (doc[-1].pos_=='PUNCT'and doc[-2]==sub[-1]):
                        text = doc.text
                        return (r_clause, sent, place, text)

            elif token.dep_ in list_rel:
                print('ADVCL')
                h = token.head
                if doc[h.head.i].lemma_ == 'be':
                    sent.insert(h.head.i - 1, nlp('that')[0])
                    rel_rec = ''
                    for i, elt in enumerate(token.subtree):
                        sent.remove(elt)
                        if i == 0:
                            rel_rec += elt.text.capitalize() + ' '
                        else:
                            rel_rec += elt.text + ' '
                    for i, elt in enumerate(sent):
                        if i == 0:
                            rel_rec += elt.text.lower() + ' '
                        else:
                            rel_rec += elt.text + ' '
                    r_clause, sent, place, t = relative_clause(rel_rec)


                rel = True
        else:
            if (token.tag_ == 'WDT' or token.tag_ == 'WP') or (token.dep_ in list_rel):
                rec = True
                break



    final_sent = [elt for elt in sent]
    for elt in r_clause[::-1]:
        final_sent.insert(place, elt)

    text = ""
    for i, elt in enumerate(final_sent):
        text += elt.text
        if i != len(final_sent) - 1 and final_sent[i + 1].pos_ != 'PUNCT':
            text += " "
    #for token in nlp(text):
     #   if(token.tag_ == 'WDT' or token.tag_ == 'WP') or (token.dep_ in list_rel):
      #      rec=True
       #     break
    if rec:
        r_clause,sent,place,text=relative_clause(text)
    return (r_clause, sent, place, text)


In [4]:
sent=['The play, which debuted last week, has been sold out every night',
      'She lives in New York, which she hates.',
      'I see a man that is tall',
      'the man that I see is tall',
      'I love the apple that I eat',
      'I’m buying a gift for the neighbor who helped me move',
      'The cat that eats the mouse is big',
      'I know which house is mine',
      'The treasure the pirates found was empty',
      'The man I see is tall']
for s in sent:
    print(f'input:  {s}')
    print(f'output:  {relative_clause(s)[3]}',end='\n\n\n')

input:  The play, which debuted last week, has been sold out every night
WDT
1
output:  The play has been sold out every night


input:  She lives in New York, which she hates.
WDT
2
output:  She hates to live in New York.


input:  I see a man that is tall
WDT
3
output:  I see a tall man


input:  the man that I see is tall
WDT
4
WDT
3
output:  I see the tall man


input:  I love the apple that I eat
WDT
5
output:  I love the apple that I eat


input:  I’m buying a gift for the neighbor who helped me move
WDT
6
output:  I ’ m buying a gift for the neighbor. The neighbor helped me move.


input:  The cat that eats the mouse is big
WDT
6
output:  The cat is big. The cat eats the mouse.


input:  I know which house is mine
WDT
7
output:  I know which house is mine


input:  The treasure the pirates found was empty
ADVCL
WDT
3
output:  The pirates found the empty treasure


input:  The man I see is tall
ADVCL
WDT
3
output:  I see the tall man


